<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-1000)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
#stock_ids

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array(['3MINDIA', 'ABBOTINDIA', 'ACC', 'ADANIENT', 'ADANIGREEN',
       'ADANIPORTS', 'ADANITRANS', 'AKZOINDIA', 'AMBUJACEM', 'APOLLOHOSP',
       'ASIANPAINT', 'ASTRAZEN', 'AUBANK', 'AUROPHARMA', 'AVANTIFEED',
       'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJAJHLDNG', 'BAJFINANCE',
       'BANDHANBNK', 'BANKBARODA', 'BATAINDIA', 'BAYERCROP', 'BERGEPAINT',
       'BHARTIARTL', 'BIOCON', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'CANBK',
       'CAPLIPOINT', 'CERA', 'CHOLAFIN', 'CIPLA', 'COALINDIA', 'COLPAL',
       'DABUR', 'DEEPAKNTR', 'DIVISLAB', 'DIXON', 'DLF', 'DMART',
       'DRREDDY', 'EICHERMOT', 'EMAMILTD', 'EQUITAS', 'EQUITASBNK',
       'ERIS', 'FEDERALBNK', 'FINCABLES', 'FINEORG', 'GAIL', 'GILLETTE',
       'GLAND', 'GLAXO', 'GODREJCP', 'GODREJIND', 'GRASIM', 'HAVELLS',
       'HCLTECH', 'HDFC', 'HDFCAMC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDALCO', 'HINDPETRO', 'HINDUNILVR', 'HONAUT', 'IBREALEST',
       'ICICIBANK', 'ICICIGI', 'ICICIPRULI', 'IGL', 'INDIANB', 'INDIGO'

In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

M    64
L    63
H    37
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0    103
1     61
Name: StarStock, dtype: int64

In [7]:
# v80 stocks
mypf['v80'].value_counts()

0    95
1    69
Name: v80, dtype: int64

In [8]:
# latest quarter
mypf['LatestQtr'].value_counts()

0    84
1    80
Name: LatestQtr, dtype: int64

In [9]:
# star + v80 stocks
mypf[mypf['v80'] == 1]['StarStock'].value_counts()

0    44
1    25
Name: StarStock, dtype: int64

In [10]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    147
1     17
Name: InPortfolio, dtype: int64

In [11]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [12]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [13]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    #stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0] 
    tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]    
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]  
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]       
    tmp['v80'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['v80'].values[0] 
    df_prec_dev = df_prec_dev.append(tmp)

In [14]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-04-12 00:00:00+05:30,17812.0,^NSEI,1.52,0.46,-0.22,18812.0,10806.0,12.49,6.0
2023-04-20 00:00:00+05:30,17624.0,^NSEI,0.16,0.60,0.03,18812.0,10806.0,14.84,7.0
2023-04-27 00:00:00+05:30,17915.0,^NSEI,1.53,0.57,-0.03,18812.0,10806.0,11.20,5.0
2023-05-05 00:00:00+05:30,18069.0,^NSEI,2.11,0.56,0.12,18812.0,10806.0,9.28,4.0
2023-05-12 00:00:00+05:30,18315.0,^NSEI,3.18,0.59,0.31,18812.0,10806.0,6.21,3.0


In [15]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [16]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [17]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'v80']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 17


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,v80
Stock,,,,,,,,,,,,,
INDIGOPNTS.NS,1301.0,-0.91,-0.69,2.11,85.10,139.0,1215.89,104.9,52.5,M,50.0,0,0
JUBLFOOD.NS,467.0,-11.15,-0.95,0.23,82.73,93.0,458.16,90.5,73.3,H,19.0,0,0
IRCTC.NS,628.0,-4.92,0.30,0.00,59.10,85.0,623.94,77.4,53.3,H,31.0,0,0
HDFCAMC.NS,1822.0,-7.45,-0.25,0.06,89.96,77.0,1856.05,38.5,27.3,M,29.0,1,1
INFY.NS,1245.0,-15.92,-0.61,-2.27,63.40,52.0,1252.12,29.0,21.5,H,26.0,1,1
DMART.NS,3678.0,-6.09,-0.38,0.46,49.15,45.0,3600.56,160.1,100.0,H,38.0,0,0
AMBUJACEM.NS,410.0,-9.89,0.44,1.84,43.73,43.0,408.85,23.9,29.1,L,-22.0,0,0
BATAINDIA.NS,1524.0,-8.11,-0.78,0.56,67.18,43.0,1501.51,126.0,61.2,H,51.0,0,1
GILLETTE.NS,4577.0,-7.04,-0.47,-0.43,77.76,30.0,4479.47,61.2,45.0,M,26.0,0,1


In [18]:
# top 5 from portfolio for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,v80
Stock,,,,,,,,,,,,,
INFY.NS,1245.0,-15.92,-0.61,-2.27,63.40,52.0,1252.12,29.0,21.5,H,26.0,1,1
JUBLFOOD.NS,467.0,-11.15,-0.95,0.23,82.73,93.0,458.16,90.5,73.3,H,19.0,0,0
AMBUJACEM.NS,410.0,-9.89,0.44,1.84,43.73,43.0,408.85,23.9,29.1,L,-22.0,0,0
BATAINDIA.NS,1524.0,-8.11,-0.78,0.56,67.18,43.0,1501.51,126.0,61.2,H,51.0,0,1
HDFCAMC.NS,1822.0,-7.45,-0.25,0.06,89.96,77.0,1856.05,38.5,27.3,M,29.0,1,1


In [19]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [20]:
# star + v80 stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['v80'] == 1)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ICICIGI.NS,1118.0,-3.93,-0.69,0.00,1620.0,1060.0,89.64,45.0,1.0,0.0,M,1,1,1
BERGEPAINT.NS,634.0,4.91,0.33,1.01,858.0,533.0,68.92,35.0,1.0,0.0,M,0,1,1
UJJIVANSFB.NS,31.0,22.60,0.00,3.70,42.0,13.0,37.93,35.0,0.0,1.0,M,1,1,1
DABUR.NS,518.0,-6.27,-0.18,-0.38,645.0,464.0,70.17,25.0,1.0,0.0,H,1,1,1
OFSS.NS,3551.0,19.90,0.81,1.59,4392.0,2442.0,43.13,24.0,1.0,0.0,M,1,1,1


In [21]:
# top 5 from star + v80 for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
DABUR.NS,518.0,-6.27,-0.18,-0.38,645.0,464.0,70.17,25.0,1.0,0.0,H,1,1,1
ICICIGI.NS,1118.0,-3.93,-0.69,0.00,1620.0,1060.0,89.64,45.0,1.0,0.0,M,1,1,1
BERGEPAINT.NS,634.0,4.91,0.33,1.01,858.0,533.0,68.92,35.0,1.0,0.0,M,0,1,1
OFSS.NS,3551.0,19.90,0.81,1.59,4392.0,2442.0,43.13,24.0,1.0,0.0,M,1,1,1
UJJIVANSFB.NS,31.0,22.60,0.00,3.70,42.0,13.0,37.93,35.0,0.0,1.0,M,1,1,1


In [22]:
# star + v80 stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['v80'] == 1)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
EQUITASBNK.NS,77.0,37.86,1.79,2.86,80.0,33.0,6.38,4.0,1.0,1.0,M,1,1,1
HINDUNILVR.NS,2623.0,2.20,0.08,0.20,2728.0,1897.0,12.64,4.0,1.0,1.0,H,1,1,1
ITC.NS,420.0,20.82,2.01,1.01,431.0,146.0,3.86,3.0,1.0,1.0,H,0,1,1
ICICIBANK.NS,944.0,7.26,1.02,1.02,953.0,333.0,1.45,1.0,1.0,1.0,H,1,1,1
BRITANNIA.NS,4616.0,13.32,1.13,0.46,4635.0,2945.0,1.12,0.0,1.0,1.0,H,1,1,1
CERA.NS,7201.0,28.66,2.14,0.62,7201.0,2229.0,0.00,0.0,1.0,1.0,H,1,1,1
POLYCAB.NS,3389.0,23.68,1.93,0.75,3389.0,793.0,0.00,0.0,1.0,0.0,M,1,1,1
TITAN.NS,2773.0,9.14,1.14,1.18,2773.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,1


In [23]:
# star stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['v80'] != 1)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ADANIGREEN.NS,896.0,-46.47,-3.41,4.09,2970.0,367.0,79.68,231.0,-1.0,-1.0,L,1,1,0
ADANIENT.NS,1965.0,-32.44,-0.76,2.11,4165.0,198.0,55.46,112.0,1.0,0.0,M,1,1,0
BANDHANBNK.NS,237.0,-3.80,-0.81,0.00,422.0,187.0,78.72,78.0,1.0,1.0,L,0,1,0
LTIM.NS,4687.0,2.39,0.59,-0.52,7439.0,2267.0,53.21,59.0,1.0,0.0,M,1,1,0
TATAPOWER.NS,207.0,-3.70,-0.47,0.00,287.0,49.0,33.61,39.0,1.0,0.0,M,1,1,0
GODREJIND.NS,472.0,7.25,0.00,0.93,644.0,354.0,59.31,36.0,0.0,1.0,L,0,1,0
BAJAJFINSV.NS,1423.0,-5.73,0.60,0.23,1907.0,542.0,35.46,34.0,0.0,0.0,M,1,1,0
SBICARD.NS,867.0,5.84,-0.12,1.19,1133.0,673.0,57.83,31.0,1.0,0.0,M,1,1,0
MCDOWELL-N.NS,796.0,-3.34,-0.12,0.52,1004.0,496.0,40.94,26.0,0.0,1.0,L,0,1,0


In [24]:
# star stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
BRITANNIA.NS,4616.0,13.32,1.13,0.46,4635.0,2945.0,1.12,0.0,1.0,1.0,H,1,1,1
CERA.NS,7201.0,28.66,2.14,0.62,7201.0,2229.0,0.00,0.0,1.0,1.0,H,1,1,1
POLYCAB.NS,3389.0,23.68,1.93,0.75,3389.0,793.0,0.00,0.0,1.0,0.0,M,1,1,1
TITAN.NS,2773.0,9.14,1.14,1.18,2773.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,1
AUBANK.NS,713.0,13.53,0.96,1.60,717.0,317.0,1.00,1.0,1.0,0.0,M,1,1,0
ICICIBANK.NS,944.0,7.26,1.02,1.02,953.0,333.0,1.45,1.0,1.0,1.0,H,1,1,1
ITC.NS,420.0,20.82,2.01,1.01,431.0,146.0,3.86,3.0,1.0,1.0,H,0,1,1
EQUITASBNK.NS,77.0,37.86,1.79,2.86,80.0,33.0,6.38,4.0,1.0,1.0,M,1,1,1
HINDUNILVR.NS,2623.0,2.20,0.08,0.20,2728.0,1897.0,12.64,4.0,1.0,1.0,H,1,1,1


In [25]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ACC.NS,1798.0,-17.67,-0.92,-0.11,2748.0,1259.0,63.80,53.0,0.0,0.0,L,0,0,0
PGHH.NS,13941.0,0.01,0.01,0.12,15778.0,9190.0,27.88,13.0,0.0,0.0,L,0,0,1
PVRINOX.NS,1447.0,-0.47,NaN,-0.83,1483.0,1446.0,97.30,2.0,1.0,-1.0,L,0,0,0
NESTLEIND.NS,21794.0,11.98,0.95,1.64,22050.0,14623.0,3.45,1.0,1.0,1.0,H,0,0,1
SIEMENS.NS,3891.0,29.70,1.67,1.51,3891.0,1143.0,0.00,0.0,1.0,1.0,H,0,0,0


In [26]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
INDUSTOWER.NS,148.0,-18.15,-1.66,-1.33,303.0,137.0,93.37,105.0,1.0,1.0,H,1,0,0
BANDHANBNK.NS,237.0,-3.80,-0.81,0.00,422.0,187.0,78.72,78.0,1.0,1.0,L,0,1,0


In [27]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 3


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
INDUSTOWER.NS,148.0,-18.15,-1.66,-1.33,303.0,137.0,93.37,105.0,1.0,1.0,H,1,0,0
BANDHANBNK.NS,237.0,-3.80,-0.81,0.00,422.0,187.0,78.72,78.0,1.0,1.0,L,0,1,0
HDFCLIFE.NS,561.0,3.25,0.00,1.18,757.0,461.0,66.22,35.0,1.0,1.0,H,1,0,0


In [28]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 15


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
CERA.NS,7201.0,28.66,2.14,0.62,7201.0,2229.0,0.00,0.0,1.0,1.0,H,1,1,1
SIEMENS.NS,3891.0,29.70,1.67,1.51,3891.0,1143.0,0.00,0.0,1.0,1.0,H,0,0,0
TITAN.NS,2773.0,9.14,1.14,1.18,2773.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,1
BRITANNIA.NS,4616.0,13.32,1.13,0.46,4635.0,2945.0,1.12,0.0,1.0,1.0,H,1,1,1
ICICIBANK.NS,944.0,7.26,1.02,1.02,953.0,333.0,1.45,1.0,1.0,1.0,H,1,1,1
NESTLEIND.NS,21794.0,11.98,0.95,1.64,22050.0,14623.0,3.45,1.0,1.0,1.0,H,0,0,1
ITC.NS,420.0,20.82,2.01,1.01,431.0,146.0,3.86,3.0,1.0,1.0,H,0,1,1
EQUITASBNK.NS,77.0,37.86,1.79,2.86,80.0,33.0,6.38,4.0,1.0,1.0,M,1,1,1
MARUTI.NS,9295.0,6.31,0.26,0.56,9528.0,6217.0,7.04,3.0,1.0,1.0,H,1,0,0


In [29]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
EQUITASBNK.NS,77.0,37.86,1.79,2.86,80.0,33.0,6.38,4.0,1.0,1.0,M,1,1,1
NESTLEIND.NS,21794.0,11.98,0.95,1.64,22050.0,14623.0,3.45,1.0,1.0,1.0,H,0,0,1
TITAN.NS,2773.0,9.14,1.14,1.18,2773.0,1090.0,0.00,0.0,1.0,1.0,H,1,1,1
COLPAL.NS,1628.0,5.01,0.19,0.98,1741.0,1262.0,23.59,7.0,1.0,1.0,H,1,1,1
DRREDDY.NS,4466.0,1.25,0.45,0.56,5451.0,3669.0,55.27,22.0,1.0,1.0,H,1,1,0
MARUTI.NS,9295.0,6.31,0.26,0.56,9528.0,6217.0,7.04,3.0,1.0,1.0,H,1,0,0
HAVELLS.NS,1306.0,6.09,0.16,0.50,1461.0,606.0,18.13,12.0,1.0,1.0,H,1,0,1
HINDUNILVR.NS,2623.0,2.20,0.08,0.20,2728.0,1897.0,12.64,4.0,1.0,1.0,H,1,1,1


In [30]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
BANDHANBNK.NS,237.0,-3.80,-0.81,0.00,422.0,187.0,78.72,78.0,1.0,1.0,L,0,1,0
INDUSTOWER.NS,148.0,-18.15,-1.66,-1.33,303.0,137.0,93.37,105.0,1.0,1.0,H,1,0,0


In [31]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] != 1) | (tmp_df['BizDurability'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 53


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ADANITRANS.NS,885.0,-63.47,-3.72,0.74,4105.0,234.0,83.18,364.0,1.0,-1.0,L,0,0,0
PEL.NS,726.0,-24.45,-4.89,-1.26,2866.0,635.0,95.92,295.0,-1.0,0.0,L,1,0,0
IBREALEST.NS,57.0,-21.46,0.00,0.00,191.0,47.0,93.06,235.0,-1.0,-1.0,L,0,0,0
ADANIGREEN.NS,896.0,-46.47,-3.41,4.09,2970.0,367.0,79.68,231.0,-1.0,-1.0,L,1,1,0
GLAND.NS,1358.0,-21.57,-3.23,0.54,4316.0,1154.0,93.55,218.0,1.0,0.0,M,0,0,0
JCHAC.NS,1106.0,-11.41,-1.76,-0.98,2837.0,1017.0,95.11,157.0,NaN,NaN,M,0,0,1
TEAMLEASE.NS,2215.0,-17.83,-2.67,-1.76,5281.0,2028.0,94.25,138.0,NaN,NaN,M,0,0,1
TASTYBITE.NS,9550.0,-11.34,-0.83,0.29,20395.0,8010.0,87.57,114.0,NaN,NaN,M,0,0,1
ADANIENT.NS,1965.0,-32.44,-0.76,2.11,4165.0,198.0,55.46,112.0,1.0,0.0,M,1,1,0


In [32]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
PVRINOX.NS,1447.0,-0.47,NaN,-0.83,1483.0,1446.0,97.3,2.0,1.0,-1.0,L,0,0,0


In [33]:
# save the output
#df_prec_dev = df_prec_dev.reset_index()
#df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/Nifty100-Upside-Analysis-ATH.csv', index=False)